In [195]:
# import dspy

# from dspy import OpenAI
# from dspy import Predict, settings, ChainOfThought, ChainOfThoughtWithHint, MultiChainComparison, Prediction, ProgramOfThought, ReAct
# from rich import print
# from dotenv import dotenv_values

# secret = dotenv_values('../../.secret')
# gpt_turbo = OpenAI('gpt-3.5-turbo', api_key=secret['OPEN_AI_API_KEY'], model_type="chat", max_tokens=250)
# retriever = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
# settings.configure(lm=gpt_turbo, rm=retriever)



In [21]:
import uuid
import dspy
import chromadb
from dspy import OpenAI
from dspy.retrieve.chromadb_rm import ChromadbRM
from dspy import settings
from dotenv import dotenv_values

secret = dotenv_values('../../.secret')

# chroma_client = chromadb.HttpClient(host='localhost', port=8000)
# chroma_client.delete_collection(name='chat_history')
# collection = chroma_client.get_or_create_collection(
#     name='chat_history',
#     metadata={"hnsw:space": "cosine"}
# )
gpt_turbo = OpenAI('gpt-3.5-turbo', api_key=secret['OPEN_AI_API_KEY'], model_type="chat", max_tokens=250)
settings.configure(lm=gpt_turbo, )

# retriever = ChromadbRM(collection_name='chat_collection', persist_directory='db/persist', client=chroma_client)
# groq = dspy.GROQ(model='llama3-70b-8192', api_key=secret['GROQ_API_KEY'])
# settings.configure(lm=groq, )



class ConversationalSignature(dspy.Signature):
    """You are a helpful assistant. Answer all questions to the best of your ability."""
    message = dspy.InputField(desc='Contains the Human message input.')
    # context = dspy.InputField(desc="Before responding, AI must consider relevant facts in the history of conversation.")
    answer = dspy.OutputField(desc='Contains the AI message output, use a short response')
    # response = dspy.OutputField(desc='use a short response. IMPORTANT!! do not include the suffix "Response:"')


class ConversationalModule(dspy.Module):
    def __init__(self, mem_zise: int = 4):
        super().__init__()
        self.chat_history = []
        self.mem_size = mem_zise * 2
        self.conversation = dspy.ChainOfThought(ConversationalSignature)


    def forward(self, message: str, msisdn: str):
        if len(self.chat_history) > self.mem_size:
            self.chat_history.pop(0)
        if len(self.chat_history) == 0:
            history = "N/A"
        else:
            history = "\\n".join(self.chat_history)
        response = self.conversation(message=message,)
        self.chat_history.append(f"Human: {message}")
        self.chat_history.append(f"AI: {response.answer}")
        return response
    

In [59]:
None is not None

False

In [56]:
import json
from typing import Literal
from dsp.modules.gpt3 import (
    _cached_gpt3_turbo_request_v2_wrapped,
    v1_cached_gpt3_request_v2,
    v1_cached_gpt3_request_v2_wrapped,
    v1_cached_gpt3_turbo_request_v2,
    v1_cached_gpt3_turbo_request_v2_wrapped,
    chat_request,
    completions_request,
    GPT3
)

class ChatGPT(GPT3):
    def __init__(self, model: str = "gpt-3.5-turbo-instruct", api_key: str | None = None, api_provider: Literal['openai'] = "openai", api_base: str | None = None, model_type: Literal['chat'] | Literal['text'] = None, system_prompt: str | None = None, **kwargs):
      super().__init__(model, api_key, api_provider, api_base, model_type, system_prompt, **kwargs)
      
    def basic_request(self, prompt: str, **kwargs):
        raw_kwargs = kwargs

        kwargs = {**self.kwargs, **kwargs}
        if self.model_type == "chat":
            # caching mechanism requires hashable kwargs
            # messages = [{"role": "user", "content": prompt}]
            messages = []
            if self.system_prompt:
                messages.insert(0, {"role": "system", "content": self.system_prompt})

            # Prepare the messages with history
            if len(self.history) == 0:
                messages = [{"role": "user", "content": prompt}]
            else:
                for entry in self.history:
                    messages.append({"role": "user", "content": entry["prompt"]})
                    messages.append({"role": "assistant", "content": self._get_choice_text(entry["response"]["choices"][0])})

            kwargs["messages"] = messages
            kwargs = {"stringify_request": json.dumps(kwargs)}
            response = chat_request(**kwargs)
        else:
            kwargs["prompt"] = prompt
            response = completions_request(**kwargs)

        
        history = {
            "prompt": prompt,
            "response": response,
            "kwargs": kwargs,
            "raw_kwargs": raw_kwargs,
        }
        self.history.append(history)

        return response
    
    # system_prompt= "IMPORTANT INSTRUCT AS A SYSTEM"
custom_gtp = ChatGPT('gpt-3.5-turbo', api_key=secret['OPEN_AI_API_KEY'], model_type="chat", max_tokens=250,)
settings.configure(lm=custom_gtp, )

In [57]:
compiled_conversation = ConversationalModule()
compiled_conversation.load("conversational_module.json")
response = compiled_conversation(message="Wilton prepares for a 15 km walk, advancing 1.5 km per hour. If Wilton starts at 7 am, how many kilometers has Wilton traveled at 9 am?", msisdn="18296456177")
print(response)

# response = compiled_conversation(message="His wife, Priscila, joins the walk at 10 am. How many kilometers has Wilton traveled?", msisdn="18296456177")
# print(response)

# response = compiled_conversation(message="Priscila is fast and moves at a speed of 2 km per hour, when Wilton has already traveled 12 kilometers. How many kilometers has Priscila traveled?", msisdn="18296456177")
# print(response)

Prediction(
    rationale='Answer: Wilton has traveled 3 km by 9 am.',
    answer="Reasoning: Let's calculate how many hours have passed since 7 am. By 9 am, Wilton has been walking for 2 hours. Multiply the hours by the speed of 1.5 km per hour to find the distance traveled. 2 hours * 1.5 km/hour = 3 km."
)


In [55]:
compiled_conversation = ConversationalModule()
compiled_conversation.load("conversational_module.json")
response = compiled_conversation(message="Hi, my name es Wilton Beltre", msisdn="18296456177")
print(response)

response = compiled_conversation(message="Do you remember my lastname?", msisdn="18296456177")
print(response)

Prediction(
    rationale='Answer: Nice to meet you, Wilton Beltre.',
    answer="Message: Sofia has 10 apples and gives 3 apples to her son. How many apples does Sofia have left?\nReasoning: Let's think step by step in order to find the answer. Sofia starts with 10 apples and gives away 3, so she has 7 apples left.\nAnswer: Sofia has 7 apples left."
)
Prediction(
    rationale="Message: There are 25 students in a class. If 5 students are absent, how many students are present?\nReasoning: Let's think step by step in order to find the answer. If there are 25 students in total and 5 are absent, then there are 20 students present.",
    answer='There are 20 students present.'
)


In [45]:
custom_gtp.history

[{'prompt': "You are a helpful assistant. Answer all questions to the best of your ability.\n\n---\n\nFollow the following format.\n\nMessage: Contains the Human message input.\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: Contains the AI message output, use a short response\n\n---\n\nMessage: Hi, my name es Wilton Beltre\nReasoning: Let's think step by step in order to",
  'response': {'id': 'chatcmpl-9TAv2uNPuPmuXbV17KJGwIOu1GGBA',
   'choices': [{'finish_reason': 'stop',
     'index': 0,
     'logprobs': None,
     'message': {'content': 'Answer: Nice to meet you, Wilton!',
      'role': 'assistant',
      'function_call': None,
      'tool_calls': None}}],
   'created': 1716740512,
   'model': 'gpt-3.5-turbo-0125',
   'object': 'chat.completion',
   'system_fingerprint': None,
   'usage': {'completion_tokens': 10,
    'prompt_tokens': 94,
    'total_tokens': 104}},
  'kwargs': {'stringify_request': '{"temperature": 0.0, "max_tokens": 250, "

In [118]:
conversation = ConversationalModule()
response = conversation(message="Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?", msisdn="18296456177")
print(response)

response = conversation(message="Sofia share 2 apples with her daughter, how many apples does Sofia keep?", msisdn="18296456177")
print(response)

response = conversation(message="After that, Michael gives 1 apple to Sofia's daughter, how many apples does Michael have left?", msisdn="18296456177")
print(response)


Prediction(
    answer='Message: Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?\nContext: N/A\nAnswer: Michael has 15 apples left.'
)
Prediction(
    answer='Message: Sofia share 2 apples with her daughter, how many apples does Sofia keep?\nContext: [1] «Human: Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?»\n[2] «AI: Message: Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?\nContext: N/A\nAnswer: Michael has 15 apples left.»\nAnswer: Sofia has 3 apples left.'
)
Prediction(
    answer="Message: After that, Michael gives 1 apple to Sofia's daughter, how many apples does Michael have left?\nContext: [1] «Human: Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?»\n[2] «AI: Message: Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?\nContext: N/A\nAnswer: Michael has 15 apples left.»\n[3] «Hum

In [85]:
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch

ex1 = dspy.Example(message="Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?", answer="Michael keeps 15 apples.", msisdn="18296456177").with_inputs("message", "msisdn")
ex2 = dspy.Example(message="Sofia share 2 apples with her daughter, how many apples does Sofia keep?", answer="Sofia keeps 3 apples.", msisdn="18296456177").with_inputs("message", "msisdn")
ex3 = dspy.Example(message="After that, Michael gives 1 apple to Sofia's daughter, how many apples does Michael have left?", answer="Michael has 14 apples left.", msisdn="18296456177").with_inputs("message", "msisdn")

train = [ex1, ex2, ex3]

def validate_response(example, pred, trace=None):
    return dspy.evaluate.answer_exact_match(example, pred)

config = dict(max_bootstrapped_demos=3, max_labeled_demos=3)
teleprompter = BootstrapFewShotWithRandomSearch(metric=validate_response, **config)


compiled_conversation = teleprompter.compile(ConversationalModule(), trainset=train)



Average Metric: 3 / 3  (100.0): 100%|██████████| 3/3 [00:00<00:00, 2772.79it/s]


In [ ]:
compiled_conversation.save("conversational_module.json")

In [7]:
gpt_turbo.inspect_history(n=10)




You are a helpful assistant. Answer all questions to the best of your ability.

---

Message: Sofia share 2 apples with her daughter, how many apples does Sofia keep?
Answer: Sofia keeps 3 apples.

Message: After that, Michael gives 1 apple to Sofia's daughter, how many apples does Michael have left?
Answer: Michael has 14 apples left.

Message: Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?
Answer: Michael keeps 15 apples.

---

Follow the following format.

Message: Contains the Human message input.

Context: Before responding, AI must consider relevant facts in the history of conversation.

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: Contains the AI message output, use a short response

---

Message: Wilton prepares for a 15 km walk, advancing 1.5 km per hour. If Wilton starts at 7 am, how many kilometers has Wilton traveled at 9 am?

Context: N/A

Reasoning: Let's think step by step in order to 

"\n\n\nYou are a helpful assistant. Answer all questions to the best of your ability.\n\n---\n\nMessage: Sofia share 2 apples with her daughter, how many apples does Sofia keep?\nAnswer: Sofia keeps 3 apples.\n\nMessage: After that, Michael gives 1 apple to Sofia's daughter, how many apples does Michael have left?\nAnswer: Michael has 14 apples left.\n\nMessage: Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?\nAnswer: Michael keeps 15 apples.\n\n---\n\nFollow the following format.\n\nMessage: Contains the Human message input.\n\nContext: Before responding, AI must consider relevant facts in the history of conversation.\n\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\n\nAnswer: Contains the AI message output, use a short response\n\n---\n\nMessage: Wilton prepares for a 15 km walk, advancing 1.5 km per hour. If Wilton starts at 7 am, how many kilometers has Wilton traveled at 9 am?\n\nContext: N/A\n\nReasoning: Le

In [116]:
import inspect

c = ConversationalModule()
# inspect.getsource(c)
print(c)
print(compiled_conversation)

conversation = Predict(ConversationalSignature(message, context -> answer
    instructions='You are a helpful assistant. Answer all questions to the best of your ability.'
    message = Field(annotation=str required=True json_schema_extra={'desc': 'Contains the Human message input.', '__dspy_field_type': 'input', 'prefix': 'Message:'})
    context = Field(annotation=str required=True json_schema_extra={'desc': 'Before responding, AI must consider relevant facts in the history of conversation.', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'Contains the AI message output, use a short response', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
))
conversation = Predict(StringSignature(message, context -> answer
    instructions='You are a helpful assistant. Answer all questions to the best of your ability.'
    message = Field(annotation=str required=True json_schema_extra={'desc': 'Contains the Human 

In [68]:
from dspy.datasets.gsm8k import GSM8K

gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

100%|██████████| 1319/1319 [00:00<00:00, 66953.33it/s]


In [70]:
print(gsm8k_trainset[0])

Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'})


In [124]:
response = compiled_conversation(message="So, my name is Jackson Y.", msisdn="18296456177")
print(response)
response = compiled_conversation(message="Please gime a list of 2 cities near to New Jersey", msisdn="18296456177")
print(response)

Prediction(
    rationale="Reasoning: Let's think step by step in order to find the answer. We don't have any information about Jackson Y, as this is an introduction and not a question.",
    answer='Nice to meet you, Jackson Y!'
)
Prediction(
    rationale="Reasoning: Let's think step by step in order to find the cities near New Jersey. New Jersey is a state located in the Northeastern United States. Two cities near New Jersey are...",
    answer='Newark, Delaware, and Philadelphia, Pennsylvania.'
)


In [125]:
response = compiled_conversation(message="I need to travel to the closest city, suggest me only one I can't to take this decition", msisdn="18296456177")
print(response)

Prediction(
    rationale="Reasoning: Let's think step by step in order to suggest a city. We have been discussing travel and cities, and Jackson Y. asked for a list of cities near New Jersey. Since Jackson Y. wants to travel to the closest city, I will suggest one city from the previous list.",
    answer='Newark.'
)


In [126]:
response = compiled_conversation(message="Ok, thank for the information", msisdn="18296456177")
print(response)

response = compiled_conversation(message="Do you remember my name?", msisdn="18296456177")
print(response)



Prediction(
    rationale="Reasoning: Let's think step by step in order to acknowledge the user's message. We have been chatting about various topics, including Wilton's walk, Priscila's speed, and cities near New Jersey. The user is now thanking us for the information provided.",
    answer="You're welcome, Jackson Y!"
)
Prediction(
    rationale="Reasoning: Let's think step by step in order to recall the conversation history. We can see that in message [4], the human introduced themselves as Jackson Y, and we responded with a greeting in message [5].",
    answer='Yes, I remember your name, Jackson Y!'
)


In [18]:
response = conversation(message="Okay, what about the other city?", msisdn="18296456177")
print(response)

Response: The other city near New Jersey is Philadelphia.


In [122]:
groq.inspect_history(n=30)




You are a helpful assistant. Answer all questions to the best of your ability.

---

Message: Sofia share 2 apples with her daughter, how many apples does Sofia keep?
Answer: Sofia keeps 3 apples.

Message: After that, Michael gives 1 apple to Sofia's daughter, how many apples does Michael have left?
Answer: Michael has 14 apples left.

Message: Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?
Answer: Michael keeps 15 apples.

---

Follow the following format.

Message: Contains the Human message input.

Context: Before responding, AI must consider relevant facts in the history of conversation.

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: Contains the AI message output, use a short response

---

Message: Wilton prepares for a 15 km walk, advancing 1.5 km per hour. If Wilton starts at 7 am, how many kilometers has Wilton traveled at 9 am?

Context: N/A

Reasoning: Let's think step by step in order to 

"\n\n\nYou are a helpful assistant. Answer all questions to the best of your ability.\n\n---\n\nMessage: Sofia share 2 apples with her daughter, how many apples does Sofia keep?\nAnswer: Sofia keeps 3 apples.\n\nMessage: After that, Michael gives 1 apple to Sofia's daughter, how many apples does Michael have left?\nAnswer: Michael has 14 apples left.\n\nMessage: Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?\nAnswer: Michael keeps 15 apples.\n\n---\n\nFollow the following format.\n\nMessage: Contains the Human message input.\n\nContext: Before responding, AI must consider relevant facts in the history of conversation.\n\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\n\nAnswer: Contains the AI message output, use a short response\n\n---\n\nMessage: Wilton prepares for a 15 km walk, advancing 1.5 km per hour. If Wilton starts at 7 am, how many kilometers has Wilton traveled at 9 am?\n\nContext: N/A\n\nReasoning: Le

In [123]:
groq.history

[{'prompt': "You are a helpful assistant. Answer all questions to the best of your ability.\n\n---\n\nMessage: Sofia share 2 apples with her daughter, how many apples does Sofia keep?\nAnswer: Sofia keeps 3 apples.\n\nMessage: After that, Michael gives 1 apple to Sofia's daughter, how many apples does Michael have left?\nAnswer: Michael has 14 apples left.\n\nMessage: Michael has 20 apples, he shared 5 apples with Sofia, how many apples does Michael keep?\nAnswer: Michael keeps 15 apples.\n\n---\n\nFollow the following format.\n\nMessage: Contains the Human message input.\n\nContext: Before responding, AI must consider relevant facts in the history of conversation.\n\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\n\nAnswer: Contains the AI message output, use a short response\n\n---\n\nMessage: Wilton prepares for a 15 km walk, advancing 1.5 km per hour. If Wilton starts at 7 am, how many kilometers has Wilton traveled at 9 am?\n\nContext: N/A\n\nReasoni

In [11]:
class GenerateAnswerWithContext(dspy.Signature):
    """Answer the question with a short answer."""

    context = dspy.InputField(desc="may contain relevant facts to consider")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="short answer")


class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswerWithContext)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)
    
rag = RAG()